# Heatmaps for carbon intensity in 2020

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import os
from os.path import join
import logging
import json

import pandas as pd
import gridemissions
from gridemissions.load import BaData
from gridemissions.viz import set_plots


In [ ]:
COLORS, PAGE_WIDTH, ROW_HEIGHT = set_plots()

In [ ]:
file_name = join(gridemissions.config["APEN_PATH"], "data", "EBA_%s.csv")
co2 = BaData(fileNm=file_name % "co2", variable="CO2")
elec = BaData(fileNm=file_name % "elec", variable="E")

In [ ]:
start = pd.to_datetime("20200101T0000Z")
end = pd.to_datetime("20210101T0000Z")
df = co2.df.loc[start:end, co2.get_cols(field="D")].copy()

In [ ]:
co2.df.index -= pd.Timedelta("7h")

In [ ]:
ba = "CISO"
co2i = pd.DataFrame({ba: (
    co2.df.loc[start:end, co2.get_cols(ba, field="D")].values.flatten()
    / elec.df.loc[start:end, elec.get_cols(ba, field="D")].values.flatten())
 for ba in co2.regions
}, index=co2.df.loc[start:end].index)

# Store index before shifting
utc_idx = co2i.index
co2i.index -= pd.Timedelta("6h")

In [ ]:
PAGE_WIDTH

In [ ]:
def plotHeatmap(
    s,
    col,
    fillValue=None,
    ynTicks=10,
    xnTicks=8,
    cbar_label=None,
    scaling=1,
    vmin=None,
    vmax=None,
    fig=None,
    ax=None,
    with_cbar=True,
    cbar_nTicks=None,
    transpose=True,
    cmap="viridis",
):
    """
        Plot a heatmap from time series data.
        Usage for dataframe df with timestamps in column "date_time" and data
        to plot in column "col" (grouped by frequency 'freq':
            df_hmap = df.groupby(pd.Grouper(key='date_time', freq=freq, axis=1))\
                .mean().fillna(method='pad',limit=fillna_limit)
            plotting.plotHeatmap(df_hmap, col)
        @param pandas.DataFrame s the data frame with the data to plot
        @param string col the column in df with the data to plot
        @param float fillValue the value to use to fill holes
        @param int nTicks the number of y ticks for the plot
        @param cbar_label string the label for the color bar
        @param float scaling a parameter to rescale the data
        @param float vmin min value for color bar
        @param float vmax max value for color bar
    """

    if fillValue is None:
        fillValue = s[col].min()
    if cbar_label is None:
        cbar_label = col
    if cbar_label == "forceNone":
        cbar_label = None

    df_heatmap = pd.DataFrame(
        {
            "date": s.index.date,
            "time": s.index.time,
            "value_col": s[col].values * scaling,
        }
    )

    if vmin is None:
        vmin = df_heatmap.value_col.min()
    if vmax is None:
        vmax = df_heatmap.value_col.max()

    df_heatmap = df_heatmap.pivot(index="date", columns="time", values="value_col")
    df_heatmap.fillna(value=fillValue, inplace=True)  # fill holes for plotting

    if fig is None:
        fig = plt.figure(figsize=(10, 10))

    if ax is None:
        ax = plt.gca()

    if transpose:
        df_heatmap = df_heatmap.transpose()

    cax = ax.pcolor(df_heatmap, cmap=cmap, vmin=vmin, vmax=vmax)

    ax.invert_yaxis()
    ynTicks = min(len(df_heatmap.index), ynTicks)
    xnTicks = min(len(df_heatmap.columns), xnTicks)
    ytickPos = range(0, len(df_heatmap.index), int(len(df_heatmap.index) / ynTicks))
    xtickPos = range(0, len(df_heatmap.columns), int(len(df_heatmap.columns) / xnTicks))

    if with_cbar:
        if cbar_label is not None:
            cb = plt.colorbar(cax, label=cbar_label, orientation="horizontal")
        else:
            cb = plt.colorbar(cax, orientation="horizontal")

    plt.xticks(
        xtickPos, [el.strftime("%m-%y") for el in df_heatmap.columns[xtickPos]]
    )
    plt.yticks(ytickPos, [el.hour for el in df_heatmap.index[ytickPos]])
    plt.ylabel("hour")
    plt.xlabel("day")
    ax.xaxis.tick_top()
    ax.xaxis.set_label_position("top")

In [ ]:
f, ax = plt.subplots(figsize=(PAGE_WIDTH, 1.5*ROW_HEIGHT))
plotHeatmap(co2i, "CISO", fig=f, ax=ax, cmap="RdYlGn_r", cbar_label="CISO - kg/MWh")
f.tight_layout()
f.savefig(join(gridemissions.config["APEN_PATH"], "CISO_in_2020_A.pdf"))

f, ax = plt.subplots(figsize=(PAGE_WIDTH, 1.5*ROW_HEIGHT))
plotHeatmap(co2i, "CISO", fig=f, ax=ax, cmap="RdYlGn_r", vmin=100, vmax=900, cbar_label="CISO - kg/MWh")
f.tight_layout()
f.savefig(join(gridemissions.config["APEN_PATH"], "CISO_in_2020_B.pdf"))

In [ ]:
n = 40
bas = co2.df.loc[start:end, co2.get_cols(field="D")].sum().sort_values(ascending=False).index[0:n]
bas = list(map(lambda x: x.split("_")[1], bas))

nrows = n // 4

f, ax = plt.subplots(nrows, 4, figsize=(1.2*PAGE_WIDTH, nrows/2* ROW_HEIGHT))
ax = ax.flatten()

for iba, ba in enumerate(bas):
    ax[iba].set_title(ba)
    with_cbar = False
    if iba == len(ax)-2:
        with_cbar = True
    plotHeatmap(co2i, ba, fig=f, ax=ax[iba], cmap="RdYlGn_r", vmin=100, vmax=900, with_cbar=with_cbar, cbar_label="kg/MWh")


for a in ax:
    a.set_yticks([])
    a.set_xticks([])
    a.set_ylabel("")
    a.set_xlabel("")
    
f.tight_layout()
f.savefig(join(gridemissions.config["APEN_PATH"], "top_40_in_2020.pdf"))